In [7]:
#pip install PyPDF2

In [8]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import PyPDF2
import openai
import streamlit as st
import requests
from bs4 import BeautifulSoup

# Set up OpenAI client
class AzureOpenAI:
    def __init__(self, api_key, api_version, azure_endpoint):
        self.api_key = api_key
        self.api_version = api_version
        self.azure_endpoint = azure_endpoint

    def embed(self, text, model):
        headers = {
            "api-key": self.api_key,
            "Content-Type": "application/json"
        }
        data = {
            "input": text,
            "model": model
        }
        response = requests.post(f"{self.azure_endpoint}/openai/text/embeddings/v1?api-version={self.api_version}", headers=headers, json=data)
        return response.json()

openai_client = AzureOpenAI(
    api_key="f21c398476b146988044b391bd5256df",
    api_version="2023-05-15",
    azure_endpoint="https://hkust.azure-api.net"
)


In [9]:
from urllib.parse import urljoin

# Scrape guidelines from HKMA website
url = "https://www.hkma.gov.hk/eng/regulatory-resources/regulatory-guides/guidelines/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a')

# Extract pdf links
pdf_links = []
for link in links:
    if link.get('href') and link.get('href').endswith('.pdf'):
        # Join relative URL with base URL
        pdf_link = urljoin(url, link.get('href'))
        pdf_links.append(pdf_link)

# Download pdf files and extract text
texts = []
for link in pdf_links:
    response = requests.get(link)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)

In [10]:
# Convert texts to embeddings
embeddings = []
for text in texts:
    embedding = openai_client.embed(text, model="text-embedding-ada-002")
    embeddings.append(embedding["embedding"])


In [11]:
# Create Streamlit app
st.title("HKMA Guidelines Q&A")

def get_answer(question):
    # Convert question to embedding
    question_embedding = openai_client.embed(question, model="text-embedding-ada-002")["embedding"]

    # Calculate cosine similarity with all embeddings
    similarities = []
    for embedding in embeddings:
        similarity = np.dot(question_embedding, embedding) / (np.linalg.norm(question_embedding) * np.linalg.norm(embedding))
        similarities.append(similarity)

    # Get index of most similar embedding
    max_index = similarities.index(max(similarities))

    # Return corresponding text as answer
    return texts[max_index]

question = st.text_input("Ask a question")
if st.button("Get answer"):
    answer = get_answer(question)
    st.write(answer)

2024-11-24 15:49:34.431 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.501 
  command:

    streamlit run C:\Users\Eric\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-24 15:49:34.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 15:49:34.506 Session state does not f

In [ ]:
#streamlit run C:\Users\Eric\anaconda3\lib\site-packages\ipykernel_launcher.py